# Pass the Hash Activity
Detects the attack technique pass the hash which is used to move laterally inside the network

## Rule Content
```
- title: Pass the Hash Activity
  id: f8d98d6c-7a07-4d74-b064-dd4a3c244528
  status: experimental
  description: Detects the attack technique pass the hash which is used to move laterally
    inside the network
  references:
  - https://github.com/iadgov/Event-Forwarding-Guidance/tree/master/Events
  author: Ilias el Matani (rule), The Information Assurance Directorate at the NSA
    (method)
  tags:
  - attack.lateral_movement
  - attack.t1075
  - car.2016-04-004
  logsource:
    product: windows
    service: security
    definition: The successful use of PtH for lateral movement between workstations
      would trigger event ID 4624, a failed logon attempt would trigger an event ID
      4625
    category: null
  detection:
    selection:
    - EventID: 4624
      LogonType: '3'
      LogonProcessName: NtLmSsp
      WorkstationName: '%Workstations%'
      ComputerName: '%Workstations%'
    - EventID: 4625
      LogonType: '3'
      LogonProcessName: NtLmSsp
      WorkstationName: '%Workstations%'
      ComputerName: '%Workstations%'
    filter:
      AccountName: ANONYMOUS LOGON
    condition: selection and not filter
  falsepositives:
  - Administrator activity
  - Penetration tests
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch([http://ip_here:9200/])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((logon_type:"3" AND logon_process_name:"NtLmSsp" AND src_host_name:"%Workstations%" AND host_name:"%Workstations%" AND (event_id:"4624" OR event_id:"4625")) AND (NOT (user_name:"ANONYMOUS\ LOGON")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()